Instalando Pacote

In [1]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 3.3 MB/s eta 0:00:00


In [2]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 83.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 101.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 23.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=9fbe31a74ec6d25c64ab37303ddbc3b97fde90840d0a71659fdceb32afe88bf3
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [3]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [4]:
perc= 5

In [5]:
from tqdm import tqdm
tqdm.pandas()
import pandas as pd

import sacrebleu


from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('sentence-transformers/paraphrase-albert-small-v2')

In [6]:
def diversidade(base):
  base['BLEUscore'] = base.progress_apply(lambda x: sacrebleu.sentence_bleu(str(x['texto']), [str(x['generated'])]), axis=1)
  base['BLEUscore_without_bp'] = base['BLEUscore'].progress_apply(lambda x: x.score/x.bp)
  return float(base['BLEUscore_without_bp'].mean())

In [7]:
def fidelidade(base, modelo):
  original_embedding = modelo.encode(base['texto'].astype('str').to_numpy())
  base['texto_embedding'] = [original_embedding[i] for i in range(original_embedding.shape[0])]

  parafrase_embedding = modelo.encode(base['generated'].astype('str').to_numpy())
  base['generated_embedding'] = [parafrase_embedding[i] for i in range(parafrase_embedding.shape[0])]

  base['similiaridade_parafrase'] = base.progress_apply(lambda x:util.cos_sim(x['texto_embedding'], x['generated_embedding']).tolist()[0][0], axis=1)

  return float(base['similiaridade_parafrase'].mean())

# Modelos

In [8]:
llms = [
       'gpt_neo_125m',
       'gpt_neo_13b',
       'gpt_neo_27b',
       'gpt_j_6b',
       'bloom_560m',
       'bloom_1b',
       'bloom_3b',
       'bloom_7b',
       'bloomz_560m',
       'bloomz_1b',
       'bloomz_3b',
       'bloomz_7b',
       'llama2_7b',
       'llama2_13b',
       'mistral_7b',
       'mistral_instruct_7b',
       'falcon_7b',
       'falcon_instruct_7b',
       ]

rota = '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/modelo/base_offensive_modelo_'

div = []
fid = []

for llm in llms:
  teste = pd.read_csv(rota+llm+'.csv', sep = ';', decimal = ',')
  div.append(diversidade(teste))
  fid.append(fidelidade(teste, model))

100%|██████████| 850/850 [00:00<00:00, 7940.01it/s]


In [9]:
data = {'llm': llms, # 'eda1', 'eda2', 'eda3', 'eda4','eda5'],
        'diversidade': div, 'fidelidade': fid,}



base_resultados = pd.DataFrame.from_dict(data)

In [10]:
base_resultados

,llm,diversidade,fidelidade
0,gpt_neo_125m,7.783933,0.313034
1,gpt_neo_13b,10.728283,0.445720
2,gpt_neo_27b,13.101743,0.504024
3,gpt_j_6b,15.375397,0.509115
4,bloom_560m,12.491513,0.450469
5,bloom_1b,8.039420,0.414099
6,bloom_3b,14.853115,0.569516
7,bloom_7b,12.449857,0.548975
8,bloomz_560m,7.249434,0.350555
9,bloomz_1b,7.232682,0.404745
